In [ ]:
library(tidyverse)

# Parameters

In [ ]:
hdf5_dname <- "../tmp2/"
expr_fname <- "../expr.csv.gz"
class_fname <- "../info.csv.gz"

In [ ]:
hdf5_dname = snakemake@input$hdf5_dname

expr_fname = snakemake@output$expression_fname
class_fname = snakemake@output$classification_fname

# Load data

In [ ]:
data <- HDF5Array::loadHDF5SummarizedExperiment(dir=hdf5_dname)

In [ ]:
head(data)

# Coarse overview

## Assay

In [ ]:
head(SummarizedExperiment::assay(data))

## Features

In [ ]:
head(SummarizedExperiment::rowRanges(data))

## Samples

In [ ]:
col.data <- SummarizedExperiment::colData(data)

In [ ]:
colnames(col.data)

# Data statistics

In [ ]:
table(col.data$definition)

In [ ]:
table(col.data$tumor_stage)

In [ ]:
table(col.data$subtype_pathologic_stage)

# Aggregate tumor stages

In [ ]:
col.data.mod <- col.data %>%
  as.data.frame %>%
  extract(tumor_stage, "tumor_stage_agg", "(.+?)[abc]?$")

In [ ]:
if ("tumor_stage" %in% colnames(col.data.mod)) {
    stop("Something went wrong")
}

In [ ]:
table(col.data.mod$tumor_stage_agg)

# Save relevant subsets

In [ ]:
col.data.mod %>%
  select(barcode, definition, tumor_stage_agg) %>%
  rename(tumor_stage=tumor_stage_agg) %>%
  write_csv(class_fname)

In [ ]:
SummarizedExperiment::assay(data) %>%
  as.data.frame %>%
  rownames_to_column("gene") %>%
  write_csv(expr_fname)